In [1]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install motor
#!{sys.executable} -m pip instalml pydantic[email]

## Setup

In [173]:
import os
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

import motor.motor_asyncio
from functools import lru_cache

from mds.config import (
    get_mongo_config
)
import uuid


In [172]:
@lru_cache()
def get_motor_client():
    mongo_config = get_mongo_config()
    mongo_config.user = "root"
    mongo_config.password = "rootpass"
    mongo_url =f"mongodb://{mongo_config.user}:{mongo_config.password}@{mongo_config.host}:{mongo_config.port}/"
    return motor.motor_asyncio.AsyncIOMotorClient(mongo_url)

mongo_config = get_mongo_config()
motor_client = get_motor_client()

db = motor_client[mongo_config.db]

In [116]:
from pydantic import BaseModel, Field, EmailStr
from bson import ObjectId
from typing import Optional, List
from fastapi.encoders import jsonable_encoder

In [117]:
class PyObjectId(ObjectId):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, v):
        if not ObjectId.is_valid(v):
            raise ValueError("Invalid objectid")
        return ObjectId(v)

    @classmethod
    def __modify_schema__(cls, field_schema):
        field_schema.update(type="string")

## Identifier Factory Functions

In [123]:
def UserId():
    return f"ark:{namespace}/user/{uuid.uuid4()}"

In [144]:
def UUIDGuidFactory():
    return f"ark:{namespace}/{uuid.uuid4()}"

In [125]:
class CompactView(BaseModel):
    guid: str = Field(alias="@id")
    metadataType: str = Field(alias="@context")
    name: str = Field(...)

## Exceptions

In [171]:
## User Exceptions
class UserExistsError(Exception):
    pass

class UserNotFoundError(Exception):
    pass


In [147]:


class MongoNotFoundError(Exception):
    pass

class MongoValidationError(Exception):
    pass

class MongoUpdateFailedError(Exception):
    pass

## MOTOR Base Functions

In [170]:
async def HydrateARKs(
    collection: motor.motor_asyncio.AsyncIOMotorCollection, 
    arks: List[str]
):
    
    pass

In [159]:
async def MotorReadByID(
    collection: motor.motor_asyncio.AsyncIOMotorCollection, 
    identifier: str
):
    result = await collection.find_one({"@id": identifier})
    if result is None:
        raise MongoNotFoundError()
    else:
        return result


In [160]:
async def MotorFindOne(
    collection: motor.motor_asyncio.AsyncIOMotorCollection,
    query: dict
):
    result = await collection.find_one(query)
    if result is None:
        raise MongoNotFoundError()
    else:
        return result

In [161]:
async def MotorUpdateOneByID(
    collection: motor.motor_asyncio.AsyncIOMotorCollection,
    identifier: str,
    update: dict
):

    update_result = await collection.update_one(
        {"@id": identifier}, 
        update
    )
    if update_result.modified_count != 1:
        raise MongoUpdateFailedError()

    updated_document = await collection.find_one({"@id": identifier})
    return updated_document

In [157]:
async def MotorInsertOne(
    collection: motor.motor_asyncio.AsyncIOMotorCollection,
    document: dict
):
    inserted_result = await collection.insert_one(document)
    return inserted_result

In [162]:
async def MotorDeleteById(
    collection: motor.motor_asyncio.AsyncIOMotorCollection,
    identifier: str
):
    # find one and delete
    document = await collection.find_one_and_delete({"@id": identifier})
    if document is None:
        raise MongoNotFoundError()
    else:
        return document

In [163]:
async def MotorList(
    collection: motor.motor_asyncio.AsyncIOMotorCollection,
    query: dict,
    length: int = 100
):
    cursor = collection.find(query)
    result_list = await cursor.to_list(length=100)
    return result_list

## User

### User Models

- CreateUserModel
    - POST Request
- StorageUserModel
    - Storage Representation of Metadata
- UpdateUserModel
    - Update request, any of the properties a user may update
- ReadUserModel
    - How User detail model is returned to a lookup request

In [166]:
class CreateUserModel(BaseModel):
    id: PyObjectId = Field(default_factory=PyObjectId, alias="_id")
    guid: str = Field(default_factory=UserId, alias="@id")
    name: str = Field(...)
    email: EmailStr = Field(...)
    password: str = Field(...)
 

    class Config:
        allow_population_by_field_name = True
        arbitrary_types_allowed = True
        json_encoders = {ObjectId: str}
        schema_extra = {
            "example": {
                "name": "Jane Doe",
                "email": "jdoe@example.com",
                "password": "test-password"
            }
        }

class UpdateUserModel(BaseModel):
    name: Optional[str] = Field(default=None)
    email: Optional[EmailStr] = Field(default=None)
    password: Optional[str] = Field(default=None)

    class Config:
        arbitrary_types_allowed = True
        json_encoders = {ObjectId: str}
        schema_extra = {
            "example": {
                "name": "Jane Doe",
                "email": "jdoe@example.com",
                "password": "test-password"
            }
        }

DefaultUserContext = {
    "@vocab": "https://schema.org/",
    "evi": "https://w3id.org/EVI#"
}



class UserMaterializedProperties(BaseModel):
    datasets: List[str] = Field(default_factory=list)
    software: List[str] = Field(default_factory=list)
    computations: List[str] = Field(default_factory=list)
    rocrates: List[str] = Field(default_factory=list)
    organizations: List[str] = Field(default_factory=list)
    projects: List[str] = Field(default_factory=list)


class ReadUserModel(UserMaterializedProperties):
    guid: str = Field(alias="@id")
    context: dict = Field(default=DefaultUserContext, alias="@type")
    metadataType: str = Field(default="Person", alias="@type")
    name: str = Field(...)
    
    
    def Hydrate(self, identifier_collection: motor.MotorCollection):
        
        pass
    

class StorageUserModel(CreateUserModel):
    active: bool = Field(default=True)
    datasets: List[str] = Field(default_factory=list)
    software: List[str] = Field(default_factory=list)
    computations: List[str] = Field(default_factory=list)
    rocrates: List[str] = Field(default_factory=list)
    organizations: List[str] = Field(default_factory=list)
    projects: List[str] = Field(default_factory=list)

    def model_post_init(self):
        pass


### USER Functions

In [138]:
async def CreateUser(user_data: CreateUserModel):
    # TODO hash password
    
    stored_user = StorageUserModel(**user_data.dict(by_alias=True))
    user_document = jsonable_encoder(stored_user)

    # does user already exist
    user_query = await db[mongo_config.user_collection].find_one(
        {"email": user_data.email}
         )
    if user_query is not None:
        raise UserExistsError()
    
    inserted_user = await db[mongo_config.user_collection].insert_one(user_document)
    return inserted_user.inserted_id


async def UpdateUser(user_id: str, user_update: UpdateUserModel):

    json_encoded_update = jsonable_encoder(user_update)
    update_json = {key: value for key, value in json_encoded_update.items() if value is not None}
    coll = db[mongo_config.user_collection]

    update_result = await coll.update_one({"@id": user_id}, {"$set": update_json})
    if update_result.modified_count != 1:
        raise MongoUpdateFailedError()

    user_document = await coll.find_one({"@id": user_id})
    if user_document is not None:
        try:
            updated_user_content = StorageUserModel(**user_document)
            return updated_user_content
        except: #validation error
            raise MongoValidationError()


async def ListUsers():
    cursor = db[mongo_config.user_collection].find()
    users = await cursor.to_list(length=100)
    return users


async def ReadUser(user_id: str):
    user_document = await db[mongo_config.user_collection].find_one({"@id": user_id})
    if user_document is None:
        raise UserNotFoundError()
    else:
        # TODO Hydrate All Arks and return UserReadModel
        # user_storage_model = StorageUserModel(**user_document)
        # user_read_model = StorageUserModel.Hydrate(collection)
        # return user_read_model
        return StorageUserModel(**user_document)


async def DeleteUser(user_id: str):
    coll = db[mongo_config.user_collection]
    user_document = await coll.find_one({"@id": user_id})
    if user_document is not None:
        try:
            user_model = StorageUserModel(**user_document)
        except: #validation error
            raise MongoValidationError()

        # update user
        update_user = coll.update_one(
            {"@id": user_id}, 
            {"$set": {"active": False, "password": ""} }
        )
        # check that record was updated
        if update_user.modified_count != 1:
            raise MongoUpdateFailedError()
        else:
            return None
    else:
        raise UserNotFoundError()

### User Tests

In [139]:


# create user
test_user = {
    "name": "Jane Doe",
    "email": "jdoe@example.com",
    "password": "test-password"
}
create_test_user = CreateUserModel(**test_user)


async def create_user():
    created_test_user_obj_id = await CreateUser(create_test_user)
    return created_test_user_obj_id

user_obj_id = await create_user()
    
# user already created
try:
    user_obj_id = await create_user()
except UserExistsError:
    pass
    


# read nonexistant user
try:
    await ReadUser("ark:99999/user/test")
except UserNotFoundError:
    print("user not found")

# update a user
name_update = UpdateUserModel(**{"name":"Jonathan Doe"})
await UpdateUser(user_id= create_test_user.guid, user_update=name_update)

    
# list users
list_users = await ListUsers()
    

user not found


## Organization Async

### Organization Models

In [146]:
class CreateOrganizationModel(BaseModel):
    guid: str = Field(default_factory=UUIDGuidFactory, alias="@id")
    type = "Organization"
    name: str = Field(...)
    description: Optional[str] = Field(...)
    url: str = Field(...)


class StorageOrganizationModel(CreateOrganizationModel):
    owner: List[str] = Field(default=[])
    members: List[str] = Field(default=[])
    projects: List[str] = Field(default=[])


class ReadOrganizationModel(CreateOrganizationModel):
    owner: List[CompactView] = Field(default=[])
    members: List[CompactView] = Field(default=[])
    projects: List[CompactView] = Field(default=[])


class UpdateOrganizationModel(BaseModel):
    name: Optional[str] = Field(default=None)
    description: Optional[str] = Field(default=None)
    members: Optional[List[str]] = Field(default=[])
    projects: Optional[List[str]] = Field(default=[])

### Organization Functions

In [ ]:
async def CreateOrganization(org_model: CreateOrganizationModel):
    
    # update user
    
    pass


async def ReadOrganization(org_id: str):
     
    pass


async def UpdateOrganization(org_id: str, org_update: UpdateOrganizationModel):
    pass


async def DeleteOrganization():

    # check if any projects are part of this organization

    # update projects to remove them from the organization

    # update users to remove them from this organization
    
    pass

### Organization Tests

## Project Async


### Project Models

### Project Functions

### Project Tests

## ROCrate Async



### ROCrate Models

### ROCrate Functions

### ROCrate Tests

## Dataset Async


### Dataset Models

### Dataset Functions

### Dataset Tests

## Computation Async

### Computation Models

### Computation Functions

### Compuatation Tests

## Software Async

### Software Models

### Software Functions

### Software Tests

## Download Async

### Download Models

### Download Functions

### Download Tests

## Notes

## Listing Elements out of motor async cursor

```
async def do_find():
    cursor = db.test_collection.find({'i': {'$lt': 5}}).sort('i')
    for document in await cursor.to_list(length=100):
        pprint.pprint(document)

loop = client.get_io_loop()
loop.run_until_complete(do_find())
```

In [45]:
new_student = await db["users"].insert_one(test_user_json)


In [47]:
new_student.acknowledged

True

In [48]:
new_student.inserted_id

'64c2bef73cae8fd48439d7ee'

In [49]:
created_student = await db["users"].find_one({"_id": new_student.inserted_id})

In [50]:
created_student

{'_id': '64c2bef73cae8fd48439d7ee',
 'name': 'Jane Doe',
 'email': 'jdoe@example.com',
 'password': 'test-password',
 'datasets': [],
 'software': [],
 'computations': [],
 'rocrates': []}